#### ID 2120

```The marketing team is evaluating the performance of their previously ran promotions.They are especially interested in comparing the sales on the first day vs. the last day of each promotion. Segment the results by promotion and find the percentage of transactions that happened on the first and last day of each. Your output should contain the promotion ID, percentage of transactions on the first day, and percentage of transactions on the last day.```

In [ ]:
%%sql
SELECT oo.promotion_id,
       100.0 * AVG(CASE WHEN date = start_date THEN 1 ELSE 0 END) AS pct_first_day,
       100.0 * AVG(CASE WHEN date = end_date THEN 1 ELSE 0 END)   AS pct_first_day
FROM online_sales_promotions AS osp
         JOIN online_orders AS oo ON osp.promotion_id = oo.promotion_id
GROUP BY oo.promotion_id

In [ ]:
df = pd.merge(online_sales_promotions, online_orders, how='inner', on='promotion_id')

# df['is_start'] = df.apply(lambda row: 1 if row['start_date'] == row['date'] else 0, axis=1)
# df['is_end'] = df.apply(lambda row: 1 if row['end_date'] == row['date'] else 0, axis=1)

df['is_start'] = (df['start_date'] == df['date']).astype('int')
df['is_end'] = (df['end_date'] == df['date']).astype('int')

result = df.groupby('promotion_id', as_index=False).agg(
    is_start_date=pd.NamedAgg(column='is_start', aggfunc=lambda series: series.mean() * 100),
    is_end_date=pd.NamedAgg(column='is_end', aggfunc=lambda series: series.mean() * 100)
)

#### ID 2121

```The marketing department is assessing the success of their promotional campaigns. You have been asked to find which products sold the most units for each promotion. Your output should contain the promotion ID, product ID, and corresponding total sales for the most successful product ID. In the case of a tie, output all results.```

In [ ]:
%%sql
WITH cte AS (SELECT promotion_id,
                    product_id,
                    SUM(units_sold)                                                AS total_sales,
                    RANK()
                    OVER (PARTITION BY promotion_id ORDER BY SUM(units_sold) DESC) AS rnk
             FROM online_orders
             GROUP BY promotion_id, product_id)
SELECT promotion_id, product_id, total_sales
FROM cte
WHERE rnk = 1

In [ ]:
df = online_orders

grouped_df = df.groupby(['promotion_id', 'product_id'], as_index=False).agg(total_units=('units_sold', 'sum'))

grouped_df['rnk'] = grouped_df.groupby('promotion_id')['total_units'].rank(method='dense', ascending=False)

grouped_df.query('rnk == 1').drop(columns='rnk')

#### ID 2122

```The VP of Sales feels that some product categories don't sell and can be completely removed from the inventory. As a first pass analysis, they want you to find what percentage of product categories have never been sold.```

In [ ]:
%%sql
SELECT (1 - AVG(sold)) * 100 AS percentage_of_unsold_categories
FROM (SELECT op.product_category,
             CASE WHEN SUM(oo.units_sold) IS NOT NULL THEN 1 ELSE 0 END AS sold
      FROM online_product_categories opc
               LEFT JOIN online_products op ON op.product_category = opc.category_id
               LEFT JOIN online_orders oo USING (product_id)
      GROUP BY op.product_category) t1

#### ID 2124

```You have been tasked with finding the top two single-channel media types (ranked in decreasing order) that correspond to the most money the grocery chain had spent on its promotional campaigns. Your output should contain the media type and the total amount spent on the advertising campaign. In the event of a tie, output all results and do not skip ranks.```

In [ ]:
%%sql
WITH total_cost_by_media_type AS (SELECT media_type,
                                         SUM(cost)                                   AS total_cost,
                                         DENSE_RANK() OVER (ORDER BY SUM(cost) DESC) AS rnk
                                  FROM online_sales_promotions
                                  GROUP BY media_type)
SELECT media_type, total_cost
FROM total_cost_by_media_type
WHERE rnk <= 2

In [ ]:
df = online_sales_promotions

grouped_df = df.groupby('media_type', as_index=False).agg(total_cost=('cost', 'sum'))

grouped_df['rnk'] = grouped_df['total_cost'].rank(method='dense', ascending=False)

grouped_df.query('rnk <= 2').drop(columns='rnk')

#### ID 2125

```Calculate and display the minimum, average and the maximum number of days it takes to process a refund for accounts opened from January 1, 2019. Group by billing cycle in months. Note: The time frame for a refund to be fully processed is from settled_at until refunded_at.```

In [ ]:
%%sql
SELECT billing_cycle_in_months,
       MIN(refunded_at - settled_at) AS min_days,
       AVG(refunded_at - settled_at) AS avg_days,
       MAX(refunded_at - settled_at) AS max_days
FROM noom_transactions AS nt
         JOIN noom_signups AS ns ON nt.signup_id = ns.signup_id
         LEFT JOIN noom_plans AS np ON ns.plan_id = np.plan_id
WHERE DATE_TRUNC('year', started_at) >= '2019-01-01'
GROUP BY billing_cycle_in_months

In [ ]:
df = pd.merge(noom_transactions, noom_signups, how='inner', on='signup_id').merge(noom_plans, how='left', on='plan_id')

df['date_diff'] = (df['refunded_at'] - df['settled_at']).dt.days

df.query('started_at.dt.year >= 2019').groupby('billing_cycle_in_months', as_index=False).agg(
    min_days=('date_diff', 'min'), avg_days=('date_diff', 'mean'), max_days=('date_diff', 'max'))

#### ID 2127

```Calculate the sales revenue for the year 2021.```

In [ ]:
%%sql
SELECT SUM(order_total) AS revenue
FROM amazon_sales
WHERE DATE_TRUNC('year', order_date) = '2021-01-01'

In [ ]:
df = amazon_sales

df['year'] = df['order_date'].dt.year

df.query('year == 2021')['order_total'].sum()

#### ID 2128

```Calculate the total revenue made per book. Output the book ID and total sales per book. In case there is a book that has never been sold, include it in your output with a value of 0.```

In [ ]:
%%sql
SELECT ab.book_id, COALESCE(SUM(quantity * unit_price), 0) AS total_sales
FROM amazon_books AS ab
         LEFT JOIN amazon_books_order_details AS abod ON ab.book_id = abod.book_id
GROUP BY ab.book_id

In [ ]:
df = pd.merge(amazon_books, amazon_books_order_details, how='left', on='book_id')
df['sales'] = df['quantity'] * df['unit_price']

df.groupby('book_id', as_index=False).agg(total_sales=('sales', 'sum'))

#### ID 2129

```You are given a list of posts of a Facebook user. Find the average number of likes.```

In [ ]:
%%sql
SELECT AVG(no_of_likes) AS avg_likes
FROM fb_posts

In [ ]:
df = fb_posts

df['no_of_likes'].mean()

#### ID 2130

```Display a list of users who took the same training lessons more than once on the same day. Output their usernames, training IDs, dates and the number of times they took the same lesson.```

In [ ]:
%%sql
WITH cte AS (SELECT u_id, training_date, training_id, COUNT(training_id) AS n_attended
             FROM training_details
             GROUP BY u_id, training_date, training_id
             HAVING COUNT(training_id) >= 2)
SELECT u_name, training_id, training_date, n_attended
FROM cte
         JOIN users_training AS ut USING (u_id)

In [ ]:
df = training_details

df.groupby(['u_id', 'training_date', 'training_id'], as_index=False).agg(n_attended=('training_id', 'count')).query(
    'n_attended >= 2').merge(users_training, how='inner', on='u_id').drop(columns='u_id')

#### ID 2132

```Given a phone log table that has information about callers' call history, find out the callers whose first and last calls were to the same person on a given day. Output the caller ID, recipient ID, and the date called.```

In [ ]:
%%sql
WITH cte AS (SELECT *,
                    FIRST_VALUE(recipient_id)
                    OVER (PARTITION BY caller_id, DATE_TRUNC('day', date_called)
                        ORDER BY date_called)                                     AS first_call,
                    FIRST_VALUE(recipient_id)
                    OVER (PARTITION BY caller_id, DATE_TRUNC('day', date_called)
                        ORDER BY date_called DESC)                                AS last_call,
                    COUNT(recipient_id)
                    OVER (PARTITION BY caller_id, DATE_TRUNC('day', date_called)) AS n_calls
             FROM caller_history)
SELECT DISTINCT caller_id, recipient_id AS recipient, date_called::DATE
FROM cte
WHERE first_call = last_call
  AND n_calls > 1

In [ ]:
df['date'] = df['date_called'].dt.date

df_1 = df.groupby(['caller_id', 'date'], as_index=False)['recipient_id'].first()

df_2 = df.groupby(['caller_id', 'date'], as_index=False)['recipient_id'].last()

pd.merge(df_1, df_2, how='inner', on=['caller_id', 'date', 'recipient_id'])

#### ID 2137

```It's the end-of-year review, and you've been tasked with identifying the city with the most profitable month in 2021. The output should provide the city, the most profitable month, and the profit.```

In [ ]:
%%sql
WITH cte AS (SELECT city,
                    EXTRACT(MONTH FROM order_date) AS p_month,
                    SUM(order_fare)                AS profit,
                    MAX(SUM(order_fare)) OVER ()   AS max_profit
             FROM lyft_orders AS lo
                      JOIN lyft_payment_details AS lpd ON lo.order_id = lpd.order_id
             WHERE DATE_TRUNC('year', order_date) = '2021-01-01'
             GROUP BY city, p_month)
SELECT city, p_month, profit
FROM cte
WHERE profit = max_profit

In [ ]:
df = pd.merge(lyft_orders, lyft_payment_details, how='inner', on='order_id')

df['year'] = df['order_date'].dt.year

df['month'] = df['order_date'].dt.month

df.query('year == 2021').groupby(['city', 'month'], as_index=False).agg(profit=('order_fare', 'sum')).nlargest(1,
                                                                                                               'profit',
                                                                                                               keep='all')

#### ID 2139

```You have been asked to calculate the average age by gender of people who filed more than 1 claim in 2021. The output should include the gender and average age rounded to the nearest whole number.```

In [ ]:
%%sql
WITH cte AS (SELECT account_id, COUNT(claim_id) AS n_claim
             FROM cvs_claims
             WHERE DATE_TRUNC('year', date_submitted) = '2021-01-01'
             GROUP BY account_id
             HAVING COUNT(claim_id) >= 2)
SELECT gender, AVG(age) AS avg_age
FROM cte
         JOIN cvs_accounts AS ca ON cte.account_id = ca.account_id
GROUP BY gender

In [ ]:
df = cvs_claims

df['year'] = df['date_submitted'].dt.year

df.query('year == 2021').groupby('account_id', as_index=False).agg(n_claims=('claim_id', 'count')).query(
    'n_claims >= 2').merge(cvs_accounts, how='inner', on='account_id').groupby('gender', as_index=False).agg(
    avg_age=('age', 'mean'))